# Crime classification in San Francisco

In [1]:
# Run these to install package with pip for the right python install

import sys
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install sklearn

You should consider upgrading via the 'C:\Users\Remi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\Remi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.



In [2]:
import pandas as pd
import seaborn as sns
#%matplotlib nbagg
import matplotlib.pyplot as plt
import numpy as np

def log_loss(p_pred,y):
    y_true=np.array(y)
    loss=0
    for i in range(len(p_pred)):
        if p_pred.shape[1]<=y_true[i]:
            loss-=np.log(1e-15)
        else:
            loss-=np.log(p_pred[i,y_true[i]]+1e-15)
    return loss/len(p_pred)

sns.set_theme()
plt.figure(figsize=(15,10))

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

In [3]:
#Read data

df = pd.read_csv('data/train.csv')
print('data loaded')

data loaded


In [4]:
#Get categories (output to find)

cat = df.Category.unique()
print(cat)
list_cat=np.copy(cat)
df.drop(['Descript','Resolution','Address'],inplace =True, axis=1)  #data useless, unsignificant, estimated useless
df.drop(df[(df.Y>60)|(df.X>-122)].index, inplace=True)  #outliers
df['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)


df.head()

['WARRANTS' 'OTHER OFFENSES' 'LARCENY/THEFT' 'VEHICLE THEFT' 'VANDALISM'
 'NON-CRIMINAL' 'ROBBERY' 'ASSAULT' 'WEAPON LAWS' 'BURGLARY'
 'SUSPICIOUS OCC' 'DRUNKENNESS' 'FORGERY/COUNTERFEITING' 'DRUG/NARCOTIC'
 'STOLEN PROPERTY' 'SECONDARY CODES' 'TRESPASS' 'MISSING PERSON' 'FRAUD'
 'KIDNAPPING' 'RUNAWAY' 'DRIVING UNDER THE INFLUENCE'
 'SEX OFFENSES FORCIBLE' 'PROSTITUTION' 'DISORDERLY CONDUCT' 'ARSON'
 'FAMILY OFFENSES' 'LIQUOR LAWS' 'BRIBERY' 'EMBEZZLEMENT' 'SUICIDE'
 'LOITERING' 'SEX OFFENSES NON FORCIBLE' 'EXTORTION' 'GAMBLING'
 'BAD CHECKS' 'TREA' 'RECOVERED VEHICLE' 'PORNOGRAPHY/OBSCENE MAT']


,Dates,Category,DayOfWeek,PdDistrict,X,Y
0,2015-05-13 23:53:00,WARRANTS,2,NORTHERN,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,2,NORTHERN,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,2,NORTHERN,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,2,NORTHERN,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,2,PARK,-122.438738,37.771541


In [5]:
#Format time data

df.Dates = pd.to_datetime(df.Dates)
df['Time'] = df.Dates.dt.hour*3600 + df.Dates.dt.minute*60 + df.Dates.dt.second
df['Hour'] = df.Dates.dt.hour
df['Month']=df.Dates.dt.month
df['Year']=df.Dates.dt.year
for key in ['Time','Hour','Month','Year']:
    df[key]=pd.to_numeric(df[key])


df['Category'].replace(to_replace=cat,value=[i for i in range(len(cat))],inplace=True)
df.head()

,Dates,Category,DayOfWeek,PdDistrict,X,Y,Time,Hour,Month,Year
0,2015-05-13 23:53:00,0,2,NORTHERN,-122.425892,37.774599,85980,23,5,2015
1,2015-05-13 23:53:00,1,2,NORTHERN,-122.425892,37.774599,85980,23,5,2015
2,2015-05-13 23:33:00,1,2,NORTHERN,-122.424363,37.800414,84780,23,5,2015
3,2015-05-13 23:30:00,2,2,NORTHERN,-122.426995,37.800873,84600,23,5,2015
4,2015-05-13 23:30:00,2,2,PARK,-122.438738,37.771541,84600,23,5,2015


In [6]:
#Cell to run if you want to have year normalized between -1 and 1
def normalize_year(data):
    years = data.Dates.dt.year.unique()
    years = sorted(years)
    a = 2/(years[-1] - years[0])
    b = -a*(years[-1]+years[0])/2
    data['Year'].replace(to_replace=years,value=[a*year +b for year in years],inplace=True)

normalize_year(df)
df.head()

,Dates,Category,DayOfWeek,PdDistrict,X,Y,Time,Hour,Month,Year
0,2015-05-13 23:53:00,0,2,NORTHERN,-122.425892,37.774599,85980,23,5,1.0
1,2015-05-13 23:53:00,1,2,NORTHERN,-122.425892,37.774599,85980,23,5,1.0
2,2015-05-13 23:33:00,1,2,NORTHERN,-122.424363,37.800414,84780,23,5,1.0
3,2015-05-13 23:30:00,2,2,NORTHERN,-122.426995,37.800873,84600,23,5,1.0
4,2015-05-13 23:30:00,2,2,PARK,-122.438738,37.771541,84600,23,5,1.0


In [7]:
def District_barycenters(data):
    Districts = data['PdDistrict'].unique()
    Bary_coord = {}
    for i in range(len(Districts)):
        test = data[['PdDistrict','X','Y']]
        district = test.loc[test['PdDistrict']==Districts[i],:]
        abs = district['X']
        ord = district['Y']
        mean_abs = abs.mean()
        mean_ord = ord.mean()
        Bary_coord[Districts[i]] = (mean_abs,mean_ord)
    data['X_Bar_District'] = data['PdDistrict']
    data['Y_Bar_District'] = data['PdDistrict']
    data.drop('PdDistrict', inplace = True, axis=1)
    data['X_Bar_District'].replace(to_replace=Districts,value=[Bary_coord[Districts[i]][0] for i in range(len(Districts))],                   inplace=True)
    data['Y_Bar_District'].replace(to_replace=Districts,value=[Bary_coord[Districts[i]][1] for i in range(len(Districts))],                   inplace=True)
District_barycenters(df)
df.head()

,Dates,Category,DayOfWeek,X,Y,Time,Hour,Month,Year,X_Bar_District,Y_Bar_District
0,2015-05-13 23:53:00,0,2,-122.425892,37.774599,85980,23,5,1.0,-122.426647,37.786379
1,2015-05-13 23:53:00,1,2,-122.425892,37.774599,85980,23,5,1.0,-122.426647,37.786379
2,2015-05-13 23:33:00,1,2,-122.424363,37.800414,84780,23,5,1.0,-122.426647,37.786379
3,2015-05-13 23:30:00,2,2,-122.426995,37.800873,84600,23,5,1.0,-122.426647,37.786379
4,2015-05-13 23:30:00,2,2,-122.438738,37.771541,84600,23,5,1.0,-122.445448,37.770299


In [8]:
def percentage_per_category(data):
    percentage_per_category = np.zeros(39)
    for c in range(39):
        #compute the percentage of category c in data
        percentage_per_category[c] = data.loc[data['Category']==c,:].count().unique()/len(data) 
    return percentage_per_category

In [9]:
# this function enables to add data in the sample in order to minimize (less than epsilon = 1e-3 here) the gap between the proportion of
# each category in the real data (df) and in the sample (dfsample)
def rebuild_data(data,sample, epsilon = 1e-3):
    real_percentage = percentage_per_category(data)
    proportion = percentage_per_category(sample)
    category = np.sort(data['Category'].unique())
    N = len(category)
    for i in range(N):
        m = category[i]
        incomplete = True
        error = np.abs(real_percentage[m] - proportion[m])
        while incomplete:
            data_cat = data.loc[data['Category'] == m,:]
            n = len(data_cat)
            index = np.random.randint(1,n)
            new_line = data_cat[index:index+1]
            sample = sample.append(new_line,ignore_index = True)
            proportion = percentage_per_category(sample)
            incomplete = real_percentage[m] - proportion[m] > epsilon
            error = np.abs(real_percentage[m] - proportion[m])
    return sample

In [10]:
#Cell to run if you want to have cyclic year

df['Sin_Year'] = np.sin(2 * np.pi * df['Time'] / (24*60*60))  # cycle rpz to explore 
df['Cos_Year'] = np.cos(2 * np.pi * df['Time'] / (24*60*60))

df['Sin_Hour'] = np.sin(2 * np.pi * df['Hour'] / 24)  # cycle rpz to explore 
df['Cos_Hour'] = np.cos(2 * np.pi * df['Hour'] / 24)

df['Sin_Month'] = np.sin(2 * np.pi * df['Month'] / 12)  # cycle rpz to explore 
df['Cos_Month'] = np.cos(2 * np.pi * df['Month'] / 12) 

df['Sin_Day'] = np.sin(2 * np.pi * df['DayOfWeek'] / 7)  # cycle rpz to explore 
df['Cos_Day'] = np.cos(2 * np.pi * df['DayOfWeek'] / 7) 

df.drop(['Time','Hour','Month','DayOfWeek'],inplace =True, axis=1)

In [35]:
dfsample = df.sample(n=100000)
dfsample = rebuild_data(df,dfsample) # new line added in order to imporve the sample
ysample=dfsample['Category']
ysample.head()
dfsample.drop('Category',inplace =True, axis=1)
dfsample.drop('Dates',inplace =True, axis=1)
dfsample.head()

,X,Y,Year,X_Bar_District,Y_Bar_District,Sin_Year,Cos_Year,Sin_Hour,Cos_Hour,Sin_Month,Cos_Month,Sin_Day,Cos_Day
0,-122.466205,37.772541,-0.833333,-122.470087,37.780208,0.004363,0.999990,0.000000,1.000000,5.000000e-01,8.660254e-01,0.433884,-0.900969
1,-122.419677,37.765851,0.166667,-122.419409,37.759961,-0.960050,-0.279829,-0.866025,-0.500000,-8.660254e-01,-5.000000e-01,-0.781831,0.623490
2,-122.410474,37.806496,0.333333,-122.409619,37.795685,-0.993572,-0.113203,-0.965926,-0.258819,1.000000e+00,6.123234e-17,-0.781831,0.623490
3,-122.411265,37.782139,0.666667,-122.412503,37.783802,-0.896873,0.442289,-0.965926,0.258819,-1.000000e+00,-1.836970e-16,-0.974928,-0.222521
4,-122.413798,37.785630,-0.333333,-122.412503,37.783802,0.500000,0.866025,0.500000,0.866025,1.224647e-16,-1.000000e+00,0.974928,-0.222521


In [36]:
# Load Library
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

#X=(dfsample-dfsample.mean())/dfsample.std()
X=dfsample
y=np.array(ysample)

# Split the training test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#  Fit a Random Forest model
clf = RandomForestClassifier(n_estimators=150,max_depth=10,min_samples_leaf=20,min_samples_split=40,max_features='sqrt',bootstrap=False,random_state=42)
#clf = MLPClassifier(hidden_layer_sizes=(100),solver='sgd',learning_rate='invscaling',random_state=1)
#clf=KNeighborsClassifier(n_neighbors=1000)
clf.fit(X_train, y_train)
p_pred = clf.predict_proba(X_test)
print(p_pred.shape)

print(log_loss(p_pred,y_test))
print(accuracy_score(y_test, np.argmax(p_pred,axis=1)))


(20025, 38)
2.4700658648379443
0.2644694132334582


In [15]:
print(clf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 40, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [32]:
from sklearn.model_selection import RandomizedSearchCV

clf = RandomForestClassifier()
# Number of trees in random forest
n_estimators =[50,100,150]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [None,5,10,15]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [20,50,100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [10,20,50]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 50, cv = 5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)




    


Fitting 5 folds for each of 50 candidates, totalling 250 fits
C:\Users\Remi\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 10, 50, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [10, 20, 50],
                                        'min_samples_split': [20, 50, 100],
                                        'n_estimators': [50, 100, 150]},
                   random_state=42, verbose=2)

In [33]:
print(rf_random.best_params_)
p_pred = rf_random.predict_proba(X_test)
print(p_pred.shape)

print(log_loss(p_pred,y_test))
print(accuracy_score(y_test, np.argmax(p_pred,axis=1)))


{'n_estimators': 150, 'min_samples_split': 50, 'min_samples_leaf': 10, 'max_features': 'sqrt', 'max_depth': 100, 'bootstrap': False}
(2020, 37)
2.7293405480175394
0.25396039603960396


In [ ]:
def cross_val_rdf(X,y,nb_range=[100],mss_range=[10]):
    n=len(nb_range)
    m=len(mss_range)
    res=np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            print((nb_range[i],mss_range[j]))
            clf=RandomForestClassifier(n_estimators=nb_range[i],min_samples_split=mss_range[j],random_state=42)
            scores=cross_val_score(clf,X,y)
            score=np.mean(scores)
            print(score)
            res[i,j]=score
    (imax,jmax)=np.unravel_index(res.argmax(), res.shape)
    return (nb_range[imax],mss_range[jmax])

cross_val_rdf(X,y,nb_range=[100,150,200],mss_range=[40,50,60])


In [ ]:
test_data = pd.read_csv('data/test.csv')
print('test data loaded')
test_data.drop(['Address'],inplace =True, axis=1)  #data useless, unsignificant
test_data['DayOfWeek'].replace(to_replace=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value=[i for i in range(0,7)],inplace=True)

test_data.Dates = pd.to_datetime(test_data.Dates)
test_data['Time'] = test_data.Dates.dt.hour*3600 + test_data.Dates.dt.minute*60 + test_data.Dates.dt.second
test_data['Hour'] = test_data.Dates.dt.hour
test_data['Month']=test_data.Dates.dt.month
test_data['Year']=test_data.Dates.dt.year
for key in ['Time','Hour','Month','Year']:
    test_data[key]=pd.to_numeric(test_data[key])
normalize_year(test_data)
District_barycenters(test_data)

test_data['Sin_Time'] = np.sin(2 * np.pi * test_data['Time'] / (24*60*60))  # cycle rpz to explore 
test_data['Cos_Time'] = np.cos(2 * np.pi * test_data['Time'] / (24*60*60))

test_data['Sin_Hour'] = np.sin(2 * np.pi * test_data['Hour'] / 24)  # cycle rpz to explore 
test_data['Cos_Hour'] = np.cos(2 * np.pi * test_data['Hour'] / 24)

test_data['Sin_Month'] = np.sin(2 * np.pi * test_data['Month'] / 12)  # cycle rpz to explore 
test_data['Cos_Month'] = np.cos(2 * np.pi * test_data['Month'] / 12) 

test_data['Sin_Day'] = np.sin(2 * np.pi * test_data['DayOfWeek'] / 7)  # cycle rpz to explore 
test_data['Cos_Day'] = np.cos(2 * np.pi * test_data['DayOfWeek'] / 7) 

test_data.drop(['Id','Time','Hour','Month','DayOfWeek','Dates'],inplace =True, axis=1)

test_data.head()

In [ ]:
list_cat=list(list_cat)
p_pred = clf.predict_proba(test_data)
if p_pred.shape[1]<len(list_cat):
    p_pred=np.concatenate((p_pred,np.zeros((len(p_pred),len(list_cat)-p_pred.shape[1]))),axis=1)

pred= pd.DataFrame (p_pred, columns = list_cat)
pred.insert(0,'Id',np.arange(len(p_pred)))

pred.head()
pred.to_csv("pred2.csv")

In [ ]:
pred.head()